Import yearly cleanup script

In [1]:
import yearly_cleanup as yc

- Input your cleanup year
- Create the new yearly folder

In [ ]:
year = yc.get_year()
yc.create_year_folder(year)

Retrieve all the subdirectories matching your cleanup year

In [ ]:
yc.get_yearly_releases(year)

Move the subdirectories matching your cleanup year to the new yearly folder

In [ ]:
yc.move_yearly_releases(yc.yearly_path, yc.release_folders)